# NightController Phase-by-Phase Test

각 Phase를 개별 셀에서 실행하여 결과를 확인합니다.

- **Phase 1**: 성찰 (Reflection)
- **Phase 2**: 계획 수립 (Planning)
- **Phase 3**: 대화 생성 (Dialogue)
- **Phase 4**: 대화 영향 분석 (Impact Analysis)
- **Phase 5**: 밤 설명 생성 (Description)

In [1]:
!git clone https://github.com/DEUS-EX-MACHlNA/demo-repository.git
%cd demo-repository/
!git checkout dev/jinhyeok
!git log
!pwd

fatal: destination path 'demo-repository' already exists and is not an empty directory.
/content/demo-repository
Already on 'dev/jinhyeok'
Your branch is up to date with 'origin/dev/jinhyeok'.
commit 53275c0657473e3706d6ead9fb1f07da17783a52 (HEAD -> dev/jinhyeok, origin/dev/jinhyeok)
Merge: 30a06ef 5d5100e
Author: Jinhyeok33 <jinhyeok2844@naver.com>
Date:   Mon Feb 2 18:01:38 2026 +0900

    Merge from raphael

commit 30a06ef0cb3aede7202359ba404e11ff04901dee
Author: Jinhyeok33 <jinhyeok2844@naver.com>
Date:   Mon Feb 2 17:56:11 2026 +0900

    feat: Everything served

commit 5d5100e3048e703abdf190e770d42391cfb2e698
Author: Raphael <lucete0972@gmail.com>
Date:   Mon Feb 2 17:55:40 2026 +0900

    feat: add endtoken in prompt

commit 2bc47375c21a43a9a03e43989deb0d34f47857b9
Author: Raphael <lucete0972@gmail.com>
Date:   Mon Feb 2 15:30:18 2026 +0900

    test: tool_turn_resolution latency check

commit a1564ea82b7e9359f1d225abb8aea4562c622975
Author: Jinhyeok33 <jinhyeok2844@naver.com>
D

In [2]:
# from google.colab import userdata
# import os

# os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
from dotenv import load_dotenv
load_dotenv("../../.env")

False

## 0. Setup — 시나리오 로드 & WorldState 생성

In [3]:
import sys, logging
from pathlib import Path

# 프로젝트 루트를 sys.path에 추가
ROOT = Path.cwd()
while not (ROOT / "app").exists() and ROOT != ROOT.parent:
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

logging.basicConfig(level=logging.INFO, format="%(name)s | %(message)s")
print(f"Project root: {ROOT}")

Project root: /content/demo-repository


In [4]:
from app.loader import ScenarioLoader
from app.models import WorldState, NPCState

# 시나리오 로드
loader = ScenarioLoader(ROOT / "scenarios")
scenario_id = loader.list_scenarios()[0]
assets = loader.load(scenario_id)

print(f"시나리오: {assets.scenario.get('title')}")
print(f"NPC: {assets.get_all_npc_ids()}")
print(f"턴 제한: {assets.get_turn_limit()}")

시나리오: 너는 이미 범인이다
NPC: ['family', 'partner', 'witness']
턴 제한: 12


In [5]:
from app.agents.memory import MemoryEntry, add_memory, MEMORY_OBSERVATION

# 테스트용 WorldState (턴 5 — 밤이 오는 시점)
world = WorldState(
    turn=5,
    npcs={
        "family": NPCState(npc_id="family", trust=2, fear=0, suspicion=0),
        "partner": NPCState(npc_id="partner", trust=1, fear=0, suspicion=2),
        "witness": NPCState(npc_id="witness", trust=0, fear=3, suspicion=1),
    },
    inventory=["casefile_brief", "pattern_analyzer", "memo_pad"],
    vars={"clue_count": 2, "identity_match_score": 1, "fabrication_score": 1},
)

# 낮 동안 쌓인 관찰 기억 시뮬레이션
fake_observations = {
    "family": [
        ("피해자 가족이 눈물을 흘리며 사건 당시를 회상했다.", 7.0),
        ("피해자 가족이 용의자에 대한 분노를 표출했다.", 6.0),
        ("수사 AI가 공감 표현을 하자 가족이 조금 안심한 듯 보였다.", 5.0),
    ],
    "partner": [
        ("동료 수사관이 증거 목록을 재검토했다.", 5.0),
        ("동료 수사관이 수사 AI의 질문 방식에 의문을 제기했다.", 8.0),
        ("동료 수사관이 패턴 분석 결과를 공유했다.", 6.0),
    ],
    "witness": [
        ("목격자가 사건 현장에서 본 것을 증언했다.", 7.0),
        ("목격자가 범인의 인상착의를 묘사했다.", 9.0),
        ("목격자가 갑자기 말을 바꾸며 불안해했다.", 8.0),
    ],
}

for npc_id, obs_list in fake_observations.items():
    for desc, imp in obs_list:
        entry = MemoryEntry.create(
            npc_id=npc_id, description=desc,
            importance_score=imp, current_turn=world.turn - 1,
            memory_type=MEMORY_OBSERVATION,
        )
        add_memory(world.npcs[npc_id].extras, entry)

# 누적 중요도를 성찰 임계값 이상으로 설정 (테스트용)
world.npcs["witness"].extras["accumulated_importance"] = 45.0

print("WorldState 준비 완료")
for npc_id, npc in world.npcs.items():
    stream = npc.extras.get("memory_stream", [])
    acc = npc.extras.get("accumulated_importance", 0)
    print(f"  {npc_id}: memories={len(stream)}, acc_importance={acc:.1f}")

WorldState 준비 완료
  family: memories=3, acc_importance=18.0
  partner: memories=3, acc_importance=19.0
  witness: memories=3, acc_importance=45.0


In [6]:
from app.agents.llm import get_llm
from importlib import reload
import sys
reload(sys.modules["app.agents.llm"])

llm = get_llm()
print(f"LLM available: {llm.available}")
print("(available=False이면 fallback 로직으로 동작합니다)")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

LLM available: True
(available=False이면 fallback 로직으로 동작합니다)


---
## Phase 1: 성찰 (Reflection)

In [7]:
from app.agents.reflection import should_reflect, perform_reflection


turn = world.turn
npc_ids = list(world.npcs.keys())
night_events: list[str] = []

print("=== Phase 1: 성찰 ===")
for npc_id in npc_ids:
    npc_state = world.npcs[npc_id]
    npc_data = assets.get_npc_by_id(npc_id)
    npc_name = npc_data["name"] if npc_data else npc_id
    persona = npc_data.get("persona", {}) if npc_data else {}

    trigger = should_reflect(npc_state.extras)
    print(f"\n[{npc_id}] should_reflect={trigger} (acc={npc_state.extras.get('accumulated_importance', 0):.1f})")

    if trigger:
        insights = perform_reflection(
            npc_id, npc_state.extras, npc_name, persona, llm, current_turn=turn,
        )
        print(f"  insights ({len(insights)}):\n" + "\n".join(f"    - {i}" for i in insights))
        night_events.append(f"{npc_name}이(가) 깊은 생각에 잠긴다.")
    else:
        print("  → 성찰 조건 미충족, skip")

print(f"\nnight_events: {night_events}")

ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 


=== Phase 1: 성찰 ===

[family] should_reflect=False (acc=18.0)
  → 성찰 조건 미충족, skip

[partner] should_reflect=False (acc=19.0)
  → 성찰 조건 미충족, skip

[witness] should_reflect=True (acc=45.0)
  insights (1):
    - 목격자은(는) 아직 답을 찾지 못했다.

night_events: ['목격자이(가) 깊은 생각에 잠긴다.']


---
## Phase 2: 계획 수립 (Planning)

In [8]:
from app.agents.planning import update_plan

scenario_title = assets.scenario.get("title", "")
turn_limit = assets.get_turn_limit()

print("=== Phase 2: 계획 수립 ===")
for npc_id in npc_ids:
    npc_state = world.npcs[npc_id]
    npc_data = assets.get_npc_by_id(npc_id)
    npc_name = npc_data["name"] if npc_data else npc_id
    persona = npc_data.get("persona", {}) if npc_data else {}

    plan = update_plan(
        npc_id, npc_name, persona, npc_state.extras,
        npc_state.trust, npc_state.fear, npc_state.suspicion,
        turn, turn_limit, scenario_title, llm,
    )
    print(f"\n[{npc_id}] plan: {plan[:100]}..." if len(plan) > 100 else f"\n[{npc_id}] plan: {plan}")

ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 


=== Phase 2: 계획 수립 ===

[family] plan: 피해자 가족은(는) 다른 사람들과 대화를 나누어 보려 한다.

[partner] plan: 동료 수사관은(는) 다른 사람들과 대화를 나누어 보려 한다.

[witness] plan: 목격자은(는) 다른 사람들과 대화를 나누어 보려 한다.


---
## Phase 3: 대화 생성 (Dialogue — Random 2명 × 2라운드)

In [9]:
import random
from app.agents.dialogue import generate_dialogue, store_dialogue_memories
from app.agents.utils import format_persona

NUM_DIALOGUE_ROUNDS = 2

all_conversations: list[tuple[str, str, list[dict[str, str]]]] = []

print("=== Phase 3: 대화 생성 ===")
for round_idx in range(NUM_DIALOGUE_ROUNDS):
    pair = random.sample(npc_ids, 2)
    npc1_id, npc2_id = pair[0], pair[1]
    s1, s2 = world.npcs[npc1_id], world.npcs[npc2_id]
    d1 = assets.get_npc_by_id(npc1_id) or {}
    d2 = assets.get_npc_by_id(npc2_id) or {}

    print(f"\n--- Round {round_idx + 1}: {d1.get('name', npc1_id)} <-> {d2.get('name', npc2_id)} ---")

    conv = generate_dialogue(
        npc1_id, d1.get("name", npc1_id), d1.get("persona", {}), s1.extras,
        s1.trust, s1.fear, s1.suspicion,
        npc2_id, d2.get("name", npc2_id), d2.get("persona", {}), s2.extras,
        s2.trust, s2.fear, s2.suspicion,
        llm, current_turn=turn,
    )
    all_conversations.append((npc1_id, npc2_id, conv))

    for utt in conv:
        print(f"  {utt['speaker']}: {utt['text']}")

    # 대화 기억 저장
    p1_str = format_persona(d1.get("persona", {}))
    p2_str = format_persona(d2.get("persona", {}))
    store_dialogue_memories(
        npc1_id, d1.get("name", npc1_id), d2.get("name", npc2_id),
        conv, s1.extras, p1_str, llm, current_turn=turn,
    )
    store_dialogue_memories(
        npc2_id, d2.get("name", npc2_id), d1.get("name", npc1_id),
        conv, s2.extras, p2_str, llm, current_turn=turn,
    )

    night_events.append(
        f"{d1.get('name', npc1_id)}과(와) {d2.get('name', npc2_id)}이(가) 대화를 나눈다."
    )

print(f"\n총 대화 라운드: {len(all_conversations)}")

ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 


=== Phase 3: 대화 생성 ===

--- Round 1: 피해자 가족 <-> 목격자 ---
  피해자 가족: ...피해자 가족은(는) 잠시 말을 아꼈다.
  목격자: ...목격자은(는) 잠시 말을 아꼈다.
  피해자 가족: ...피해자 가족은(는) 잠시 말을 아꼈다.
  목격자: ...목격자은(는) 잠시 말을 아꼈다.

--- Round 2: 동료 수사관 <-> 목격자 ---
  동료 수사관: ...동료 수사관은(는) 잠시 말을 아꼈다.
  목격자: ...목격자은(는) 잠시 말을 아꼈다.
  동료 수사관: ...동료 수사관은(는) 잠시 말을 아꼈다.
  목격자: ...목격자은(는) 잠시 말을 아꼈다.

총 대화 라운드: 2


---
## Phase 4: 대화 영향 분석 (Impact Analysis)

In [10]:
from app.agents.dialogue import analyze_conversation_impact
from typing import Any

night_delta: dict[str, Any] = {
    "turn_increment": 1,
    "npc_stats": {},
    "vars": {},
}

print("=== Phase 4: 대화 영향 분석 ===")
for npc1_id, npc2_id, conv in all_conversations:
    d1 = assets.get_npc_by_id(npc1_id) or {}
    d2 = assets.get_npc_by_id(npc2_id) or {}
    changes = analyze_conversation_impact(
        npc1_id, d1.get("name", npc1_id), d1.get("persona", {}),
        npc2_id, d2.get("name", npc2_id), d2.get("persona", {}),
        conv, llm,
    )
    print(f"\n{d1.get('name')} <-> {d2.get('name')}:")
    for npc_id, stat_changes in changes.items():
        print(f"  {npc_id}: {stat_changes}")
        if stat_changes:
            night_delta["npc_stats"].setdefault(npc_id, {})
            for stat, val in stat_changes.items():
                night_delta["npc_stats"][npc_id][stat] = (
                    night_delta["npc_stats"][npc_id].get(stat, 0) + val
                )

print(f"\nnight_delta: {night_delta}")

ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 


=== Phase 4: 대화 영향 분석 ===

피해자 가족 <-> 목격자:
  family: {}
  witness: {}

동료 수사관 <-> 목격자:
  partner: {}
  witness: {}

night_delta: {'turn_increment': 1, 'npc_stats': {}, 'vars': {}}


---
## Phase 5: 밤 설명 생성 (Description)

In [11]:
from app.agents.generative_night import NightController

night_description = NightController._generate_description(
    world, assets, all_conversations, night_events, llm,
)

print("=== Phase 5: 밤 설명 ===")
print(night_description)

ERROR:app.agents.llm:LLM generation failed: 


=== Phase 5: 밤 설명 ===
밤이 깊어간다. 진실과 조작의 경계가 흐려진다. 목격자이(가) 깊은 생각에 잠긴다.


---
## Final: NightResult 조립

In [12]:
from app.models import NightResult

night_conversation: list[list[dict[str, str]]] = [
    conv for _, _, conv in all_conversations
]

result = NightResult(
    night_delta=night_delta,
    night_conversation=night_conversation,
    night_description=night_description,
)

print("=== NightResult ===")
print(f"night_delta: {result.night_delta}")
print(f"\nnight_description:\n  {result.night_description}")
print(f"\nnight_conversation ({len(result.night_conversation)} rounds):")
for i, conv in enumerate(result.night_conversation):
    print(f"\n  [Round {i+1}] ({len(conv)} utterances)")
    for utt in conv:
        print(f"    {utt['speaker']}: {utt['text']}")

=== NightResult ===
night_delta: {'turn_increment': 1, 'npc_stats': {}, 'vars': {}}

night_description:
  밤이 깊어간다. 진실과 조작의 경계가 흐려진다. 목격자이(가) 깊은 생각에 잠긴다.

night_conversation (2 rounds):

  [Round 1] (4 utterances)
    피해자 가족: ...피해자 가족은(는) 잠시 말을 아꼈다.
    목격자: ...목격자은(는) 잠시 말을 아꼈다.
    피해자 가족: ...피해자 가족은(는) 잠시 말을 아꼈다.
    목격자: ...목격자은(는) 잠시 말을 아꼈다.

  [Round 2] (4 utterances)
    동료 수사관: ...동료 수사관은(는) 잠시 말을 아꼈다.
    목격자: ...목격자은(는) 잠시 말을 아꼈다.
    동료 수사관: ...동료 수사관은(는) 잠시 말을 아꼈다.
    목격자: ...목격자은(는) 잠시 말을 아꼈다.


---
## (참고) NightController.run() 통합 실행

위의 Phase들을 한번에 실행하려면:

In [13]:
from app.agents.generative_night import NightController
from app.models import WorldState, NPCState
from app.agents.memory import MemoryEntry, add_memory, MEMORY_OBSERVATION

# 새 WorldState (이전 Phase들에서 extras가 변경되었으므로 새로 생성)
world2 = WorldState(
    turn=5,
    npcs={
        "family": NPCState(npc_id="family", trust=2, fear=0, suspicion=0),
        "partner": NPCState(npc_id="partner", trust=1, fear=0, suspicion=2),
        "witness": NPCState(npc_id="witness", trust=0, fear=3, suspicion=1),
    },
    inventory=["casefile_brief", "pattern_analyzer", "memo_pad"],
    vars={"clue_count": 2, "identity_match_score": 1, "fabrication_score": 1},
)

for npc_id, obs_list in fake_observations.items():
    for desc, imp in obs_list:
        entry = MemoryEntry.create(
            npc_id=npc_id, description=desc,
            importance_score=imp, current_turn=4,
            memory_type=MEMORY_OBSERVATION,
        )
        add_memory(world2.npcs[npc_id].extras, entry)
world2.npcs["witness"].extras["accumulated_importance"] = 45.0

controller = NightController()
result2 = controller.run(world2, assets)

print(f"night_delta: {result2.night_delta}")
print(f"night_description: {result2.night_description}")
print(f"conversation rounds: {len(result2.night_conversation)}")

ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.agents.llm:LLM generation failed: 
ERROR:app.

night_delta: {'turn_increment': 1, 'npc_stats': {}, 'vars': {}}
night_description: 시간이 흐른다. 당신의 질문들이 세계를 바꾸고 있다. 목격자이(가) 깊은 생각에 잠긴다.
conversation rounds: 2
